In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [3]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t', header = None)
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t', header = None)
test_set = np.array(training_set, dtype = 'int')

In [4]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [5]:
training_set = convert(training_set)
test_set = convert(test_set)

training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [8]:
neurons_1 = 20
neurons_2 = 10

class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, neurons_1)
        self.fc2 = nn.Linear(neurons_1, neurons_2)
        self.fc3 = nn.Linear(neurons_2, neurons_1)
        self.fc4 = nn.Linear(neurons_1, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [9]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [12]:
nb_epochs = 200

for epoch in range(1, nb_epochs + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input_user = Variable(training_set[id_user]).unsqueeze(0)
        target = input_user.clone()
        if torch.sum(target.data > 0) > 0:
            output_user = sae(input_user)
            target.require_grad = False
            output_user[target == 0] = 0
            loss = criterion(output_user, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss/s))

epoch: 1 loss: tensor(1.7650)
epoch: 2 loss: tensor(1.0965)
epoch: 3 loss: tensor(1.0535)
epoch: 4 loss: tensor(1.0384)
epoch: 5 loss: tensor(1.0309)
epoch: 6 loss: tensor(1.0265)
epoch: 7 loss: tensor(1.0237)
epoch: 8 loss: tensor(1.0220)
epoch: 9 loss: tensor(1.0209)
epoch: 10 loss: tensor(1.0194)
epoch: 11 loss: tensor(1.0189)
epoch: 12 loss: tensor(1.0184)
epoch: 13 loss: tensor(1.0179)
epoch: 14 loss: tensor(1.0175)
epoch: 15 loss: tensor(1.0170)
epoch: 16 loss: tensor(1.0168)
epoch: 17 loss: tensor(1.0168)
epoch: 18 loss: tensor(1.0163)
epoch: 19 loss: tensor(1.0162)
epoch: 20 loss: tensor(1.0163)
epoch: 21 loss: tensor(1.0159)
epoch: 22 loss: tensor(1.0160)
epoch: 23 loss: tensor(1.0157)
epoch: 24 loss: tensor(1.0158)
epoch: 25 loss: tensor(1.0155)
epoch: 26 loss: tensor(1.0156)
epoch: 27 loss: tensor(1.0154)
epoch: 28 loss: tensor(1.0150)
epoch: 29 loss: tensor(1.0131)
epoch: 30 loss: tensor(1.0111)
epoch: 31 loss: tensor(1.0105)
epoch: 32 loss: tensor(1.0075)
epoch: 33 loss: t

In [14]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input_user = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output_user = sae(input_user)
        target.require_grad = False
        output_user[target == 0] = 0
        loss = criterion(output_user, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: ' + str(test_loss/s))

test loss: tensor(0.9132)
